In [1]:
import sklearn as sk
import pandas as pd
import xgboost as xgb

from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import ADASYN
from imblearn.combine import SMOTETomek
from imblearn.pipeline import Pipeline


In [2]:
#arquivo treino

#caminho para rodar google colab
#arq_treinamento = pd.read_csv('treino.csv')
#arq_teste = pd.read_csv('teste.csv')


#caminhos para rodar localmente
arq_treinamento = pd.read_csv('arquivos/brutos/treino.csv')



def arq_treinamento_info():
    
    print(arq_treinamento.head())
    print(arq_treinamento.info())
    print(arq_treinamento.describe())
    print(arq_treinamento.isnull().sum())


#definindo target e características arquivo de treinamento
arq_treinamento_x = arq_treinamento.drop(columns=['target', 'id'])

arq_treinamento_y = arq_treinamento['target']



#train_test_split dos dados de teste

X_train, X_test, y_train, y_test = train_test_split(arq_treinamento_x, arq_treinamento_y, test_size=0.2, random_state=42)

#targets estao desbalanceados
#arq_treinamento_y.value_counts()
y_train.value_counts()

def split_info():
    print("Dados de treinamento (X_train):")
    print(X_train)
    print("Labels de treinamento (y_train):")
    print(y_train)
    print("Dados de teste (X_test):")
    print(X_test)
    print("Labels de teste (y_test):")
    print(y_test)

In [3]:
#balanceamento SMOTETomek ADASYN

random_state_42 = 42

balanceador = Pipeline([
    ('adasyn', ADASYN(random_state=random_state_42)),
    ('smote_tomek_balanceamento', SMOTETomek(random_state=random_state_42))
])

X_rebalanceado_smote_tomek_adasyn, y_rebalanceado_smote_tomek_adasyn = balanceador.fit_resample(
    X_train, y_train)

#print(X_rebalanceado_smote_tomek_adasyn)
#print(y_rebalanceado_smote_tomek_adasyn)
#y_rebalanceado_smote_tomek_adasyn.value_counts()

In [4]:
#Treinando o modelo

#XGBoost n_iter=10-RandomSearchCV
#F1-Score mais alta:  0.765687

modelo_final_XGBoost = XGBClassifier(
        objective="multi:softmax",
        random_state=random_state_42,
        subsample=1.0, 
        n_estimators=500, 
        max_depth=20, 
        learning_rate=0.2, 
        colsample_bytree=0.8
    )

modelo_final_XGBoost.fit(X_rebalanceado_smote_tomek_adasyn, y_rebalanceado_smote_tomek_adasyn)






XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.2, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=20,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=500,
              n_jobs=None, num_parallel_tree=None, ...)

In [5]:
#arquivo de teste
arq_teste = pd.read_csv('arquivos/brutos/teste.csv')

arq_teste_ids = arq_teste['id']

arq_teste_X = arq_teste.drop(columns=['id'])








In [6]:
#predição e geração do arquivo resposta
arq_teste_pred = modelo_final_XGBoost.predict(arq_teste_X)

arq_teste_saida = pd.DataFrame({'id':arq_teste_ids, 'target': arq_teste_pred})

arq_teste_saida.to_csv('arquivos/final/ezequiel_melo_nogueira_3_fase_onia_resposta_teste.csv', index=False)


In [7]:
#verificando o arquivo resposta

arq_resposta = pd.read_csv('arquivos/final/ezequiel_melo_nogueira_3_fase_onia_resposta_teste.csv')

def arq_info(arq):
    
    print(arq.head())
    #print(arq.info())
    #print(arq.describe())
    #print(arq.isnull().sum())

print('arquivo resposta:', arq_info(arq_resposta))
         
print('arquivo teste:', arq_info(arq_teste))

      id  target
0   2142       4
1   3846       0
2   6177       4
3  11779       0
4   1593       0
arquivo resposta: None
      id      col_0     col_1     col_2      col_3      col_4      col_5  \
0   2142  37.555079  3.205281  1.583984  11.330132  32.542511   6.581602   
1   3846  28.933095  3.905861  2.171001  14.644065  26.406560   9.515010   
2   6177  37.082903  3.259937  1.887675  12.313557  29.368109   6.533770   
3  11779  24.011327  4.599539  2.102587  17.349264  27.121683  12.231295   
4   1593  31.403592  2.543982  0.533288  14.778941  43.525198   5.225097   

      col_6     col_7     col_8     col_9     col_10     col_11    col_12  
0  5.524091  5.202632  3.288405  3.009713  14.706292  20.530879  2.889453  
1  4.023027  6.528586  4.202300  2.281440  18.454367  14.862851  3.183574  
2  5.088085  5.913968  3.925079  2.619016  16.717026  16.871735  3.512345  
3  2.664319  5.226405  5.232794  2.996656  14.773491   9.133299  3.739929  
4  7.100292  4.516120  2.753415  3.386